USING PYTORCH

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Using GPU from google colab
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
cuda0 = torch.device('cuda:0')

1
Tesla P100-PCIE-16GB


In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
#Hyperparameters
batch_size=16
no_of_epoches =25
learning_rate = 0.001
momentum=0.9
hidden_layer_1 = 512
hidden_layer_2 = 128

In [0]:
# Loading the dataset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

classes = ['airplane', 'automobile', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [0]:
class Net(nn.Module):
  def __init__(self,hidden_layer_1, hidden_layer_2):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1) # output size = 32x32
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1) # output size = 32x32
    self.batch1 = nn.BatchNorm2d(32)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # output size = 16x16

    self.conv3 = nn.Conv2d(in_channels=32,out_channels=64, kernel_size=3, stride=1,padding=1) # output size = 16x16
    self.conv4 = nn.Conv2d(in_channels=64,out_channels=64, kernel_size=3, stride=1,padding=1) # output size = 16x16
    self.batch2 = nn.BatchNorm2d(64) 
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # output size = 8x8

    self.conv5 = nn.Conv2d(in_channels=64,out_channels=128, kernel_size=3, stride=1, padding=1) # output size = 8x8
    self.batch3 = nn.BatchNorm2d(128) 
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # output size = 4x4

    self.lin1 = nn.Linear(128*4*4, hidden_layer_1)
    self.batch4 =nn.BatchNorm1d(hidden_layer_1)
    self.lin2 = nn.Linear(hidden_layer_1,hidden_layer_2)
    self.batch5 =nn.BatchNorm1d(hidden_layer_2)
    self.lin3 = nn.Linear(hidden_layer_2,10)

  def forward(self, x):
    x = x.float()
    x = self.conv1(x)   # Conv layer 1
    x = self.conv2(x)   # Conv layer 2
    x = F.relu(x)       # Relu activation function
    x = self.batch1(x)  # Batch normalization Layer
    x = self.pool1(x)   # Max pooling Layer

    x = self.conv3(x)   # Conv layer 3
    x = self.conv4(x)   # Conv layer 4
    x = F.relu(x)       # Relu activation function
    x = self.batch2(x)  # Batch normalization Layer
    x = self.pool2(x)   # Max pooling Layer

    x = self.conv5(x)   # Conv Layer 5
    x = F.relu(x)       # Relu activation function
    x = self.batch3(x)  # Batch normalization Layer
    x = self.pool3(x)   # Max pooling Layer

    x = x.view(-1, 128*4*4) # Flattening output of conv neural network into 1-dim
    x = self.lin1(x)        # fully connected layer 1
    x = F.relu(x)           # Relu activation function

    x = self.lin2(x)        # fully connected layer 1
    x = F.relu(x)           # Relu activation function

    x = self.lin3(x)        # fully connected layer 1

    return x


**Output layer size formula** : [(n-f+2p)/s]+1

where : *nxn* input, *fxf* filter size, *p* padding, *s* stride


In [0]:
class train():
  def __init__(self,net,no_of_epoches):
    self.net = net.to(cuda0)
    self.no_of_epoches = no_of_epoches

  def train_network(self):
    print("Training started...")
    for epoch in range(self.no_of_epoches):
      self.total_loss = 0.0
      for i,images in enumerate(trainloader,0):
        image, label = images
        image = image.to(cuda0)
        label = label.to(cuda0)
        optimizer.zero_grad()
        output = self.net(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        self.total_loss += loss.item() 
      print("EPOCH : "+str(epoch+1)+"| Loss :"+str(self.total_loss))
    print("Training completed !")

In [0]:
class evaluate():
  def __init__ (self, net, classes):
    self.net = net.to(cuda0)
    self.predicted = torch.tensor([0], device=cuda0)
    self.true_classes = torch.tensor([0], device= cuda0)
    self.correct_classes =0
    self.classes = classes
    self.dic = {}

  def test(self):
    print("Evaluation started...")

    for i, test_images in enumerate(testloader):
      image, label = test_images
      image = image.to(cuda0)
      label = label.to(cuda0)
      outputs = self.net(image)
      _, pred = torch.max(outputs,1)
      self.predicted = torch.cat([self.predicted,pred],0)
      self.true_classes = torch.cat([self.true_classes,label],0)

    self.predicted = self.predicted[1:]
    self.true_classes = self.true_classes[1:]
    self.check_performance_overall()

  def check_performance_overall(self):
    for i in range(self.predicted.shape[0]):
      if self.predicted[i] == self.true_classes[i]:
        self.correct_classes +=1
        if self.predicted[i].item() not in self.dic.keys():
          self.dic[self.predicted[i].item()] = 1
        else:
          self.dic[self.predicted[i].item()] += 1 
        
    print("Total Accuracy in % : ", self.correct_classes*100/self.predicted.shape[0])
    self.check_performance_per_class()

  def check_performance_per_class(self):
    data = np.divide(list(self.dic.values()),1000)   
    dataframe = pd.DataFrame(data=data, index=self.classes, columns=['Accuracy'])
    print("\nPerformance per class")
    print(dataframe)


In [0]:
#defining CNN 
net = Net(hidden_layer_1,hidden_layer_2)
#defining loss function : cross entropy loss
criterion = nn.CrossEntropyLoss()
#defining optimizer : Stochastic gradient descent
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

In [83]:
# Training the network
train_net=train(net, no_of_epoches)
train_net.train_network()

Training started...
EPOCH : 1| Loss :3646.927947193384
EPOCH : 2| Loss :2334.7086061686277
EPOCH : 3| Loss :1806.8390987217426
EPOCH : 4| Loss :1414.6074264720082
EPOCH : 5| Loss :1084.992445960641
EPOCH : 6| Loss :793.9396838396788
EPOCH : 7| Loss :552.8699273765087
EPOCH : 8| Loss :404.59053230285645
EPOCH : 9| Loss :308.4038810580969
EPOCH : 10| Loss :222.6740596666932
EPOCH : 11| Loss :174.16632789373398
EPOCH : 12| Loss :139.53132029622793
EPOCH : 13| Loss :104.1984837949276
EPOCH : 14| Loss :99.91046011447906
EPOCH : 15| Loss :98.25516854226589
EPOCH : 16| Loss :72.6279126405716
EPOCH : 17| Loss :58.36492218077183
EPOCH : 18| Loss :51.91776901483536
EPOCH : 19| Loss :54.34536826610565
EPOCH : 20| Loss :46.72621840238571
EPOCH : 21| Loss :43.99982608854771
EPOCH : 22| Loss :37.37608578801155
EPOCH : 23| Loss :25.077120512723923
EPOCH : 24| Loss :17.988054871559143
EPOCH : 25| Loss :13.527503043413162
Training completed !


In [84]:
# Evaluating the network
evaluate_net = evaluate(net,classes)
evaluate_net.test()

Evaluation started...
Total Accuracy in % :  80.24

Performance per class
            Accuracy
airplane       0.758
automobile     0.903
bird           0.880
cat            0.868
deer           0.865
dog            0.845
frog           0.669
horse          0.847
ship           0.788
truck          0.601


In [0]:
#Save the network
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [87]:
#load network
PATH = './cifar_net.pth'
net_loaded = Net(hidden_layer_1,hidden_layer_2)
net_loaded.load_state_dict(torch.load(PATH))
print("Model Loaded sucessfully.")
evaluate_net = evaluate(net_loaded.to(cuda0),classes)
evaluate_net.test()

Model Loaded sucessfully.
Evaluation started...
Total Accuracy in % :  80.26

Performance per class
            Accuracy
airplane       0.852
automobile     0.753
bird           0.607
cat            0.902
deer           0.879
dog            0.790
frog           0.665
horse          0.863
ship           0.869
truck          0.846
